In [80]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
from xgboost import XGBRegressor
import re

In [84]:
#add advanced stats from fantasy pros
def cleanFP(dfwr):
    dfwr['Player'] = dfwr['Player'].str.split("(").str[0]
    dfwr = dfwr.drop(columns=['AIR', 'AIR/R'])
    return dfwr
#add basic stats from pro football reference
def cleanPFR(dfwr):
    dfwr['Player'] = dfwr['Player'].str.split("(").str[0]
    dfwr['YDS'] = dfwr['YDS'].str.replace(',', '').astype(float)
    #dfwr['Fpts/G'] = dfwr.apply(lambda row: (row.REC + row.YDS / 10 + row.TD * 6) / row.G, axis=1)
    dfwr = dfwr.drop(columns=['AIR', 'AIR/R'])
    return dfwr

dfwr22A = pd.read_excel(r"C:\Users\luker\Downloads\Python\WR22.xlsx", 'WR22A')
dfwr22A = cleanFP(dfwr22A)
dfwr22B = pd.read_excel(r"C:\Users\luker\Downloads\Python\WR22.xlsx", 'WR22B')
#dfwr22B = cleanPFR(dfwr22B)
dfwr22A

,Rank,Player,G,REC,YDS,Y/R,YBC,YBC/R,YAC,YAC/R,...,% TM,CATCHABLE,DROP,RZ TGT,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,LNG
0,1,Justin Jefferson,17,128,1809,14.1,1185,9.3,624,4.9,...,0.287,134,5,28,74,28,14,6,1,64
1,2,Davante Adams,17,100,1516,15.2,1023,10.2,493,4.9,...,0.323,109,8,22,61,24,14,7,2,60
2,3,Tyreek Hill,17,119,1710,14.4,1226,10.3,484,4.1,...,0.302,127,8,9,69,25,9,7,4,64
3,4,A.J. Brown,17,88,1496,17.0,953,10.8,543,6.2,...,0.287,95,6,16,58,23,13,6,3,78
4,5,Stefon Diggs,17,110,1455,13.2,1023,9.3,432,3.9,...,0.281,117,7,23,57,17,10,6,2,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,192,Bryan Edwards,4,3,15,5.0,6,2.0,9,3.0,...,0.013,5,0,3,1,0,0,0,0,10
192,193,Austin Trammell,2,2,13,6.5,13,6.5,0,0.0,...,0.008,2,0,0,0,0,0,0,0,8
193,194,Montrell Washington,15,4,2,0.5,-11,-2.8,13,3.3,...,0.017,5,1,0,0,0,0,0,0,4
194,195,Lance McCutcheon,4,0,0,0.0,0,0.0,0,0.0,...,0.010,0,0,1,0,0,0,0,0,0
